In [1]:
%matplotlib inline
import utils#; reload(utils)
from utils import *
from __future__ import division, print_function


import os, json
from glob import glob
import numpy as np
from scipy import misc, ndimage
from scipy.ndimage.interpolation import zoom

from keras.utils.data_utils import get_file
from keras import backend as K
from keras.layers.normalization import BatchNormalization
from keras.utils.data_utils import get_file
from keras.models import Sequential
from keras.layers.core import Flatten, Dense, Dropout, Lambda
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.layers.pooling import GlobalAveragePooling2D
from keras.optimizers import SGD, RMSprop, Adam
from keras.preprocessing import image

from keras.callbacks import ModelCheckpoint


WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

Using gpu device 0: Tesla K80 (CNMeM is disabled, cuDNN 5103)
Using Theano backend.


In [2]:
#path = "data/fish/sample/"
path = "data/cropped_nof_excl/"
batch_size=32
target_size = (450, 450)

early_stopping  = keras.callbacks.EarlyStopping(monitor='val_loss', patience=1)


In [3]:
(val_classes, trn_classes, val_labels, trn_labels, 
    val_filenames, filenames, test_filenames) = get_classes(path)

Found 2912 images belonging to 7 classes.
Found 400 images belonging to 7 classes.
Found 12153 images belonging to 1 classes.


In [4]:
raw_filenames = [f.split('/')[-1] for f in filenames]
raw_test_filenames = [f.split('/')[-1] for f in test_filenames]
raw_val_filenames = [f.split('/')[-1] for f in val_filenames]

###  End-to-end convnet without pre-computations

In [8]:
gen = image.ImageDataGenerator(rotation_range=10, width_shift_range=0.05, zoom_range=0.05,
                                      channel_shift_range=5, height_shift_range=0.05, shear_range=0.05,
                                      fill_mode='nearest',horizontal_flip=True)

In [15]:
train_flow = gen.flow_from_directory(path+"train", target_size=target_size,
            class_mode="categorical", shuffle=True, batch_size=batch_size)

val_flow = image.ImageDataGenerator().flow_from_directory(path+"valid", target_size=target_size, batch_size=batch_size,
                                                   class_mode='categorical', shuffle=True)

Found 2912 images belonging to 7 classes.
Found 400 images belonging to 7 classes.


In [9]:
nf=128; p=0.

def get_additional_layers(conv_layers):
    return [
        BatchNormalization(axis=1, input_shape=conv_layers[-1].output_shape[1:]),
        Convolution2D(nf,3,3, activation='relu', border_mode='same'),
        BatchNormalization(axis=1),
        MaxPooling2D(),
        Convolution2D(nf,3,3, activation='relu', border_mode='same'),
        BatchNormalization(axis=1),
        MaxPooling2D(),
        Convolution2D(nf,3,3, activation='relu', border_mode='same'),
        BatchNormalization(axis=1),
        MaxPooling2D((1,2)),
        Convolution2D(7,3,3, border_mode='same'),
        Dropout(p),
        GlobalAveragePooling2D(),
#         GlobalMaxPooling2D(),
        Activation('softmax')
    ]



In [10]:
model = Vgg16BN(target_size).model #(360, 640)
model.pop()

for layer in model.layers: #Freeze feature layers
    layer.trainable = False
    
    
    
conv_layers,_ = split_at(model, Convolution2D)

for l in get_additional_layers(conv_layers):
    model.add(l)
    
print (model.input_shape, model.output_shape)
model.compile(Adam(lr=0.001), 'categorical_crossentropy', metrics=['accuracy'])


/home/ubuntu/anaconda2/lib/python2.7/site-packages/keras/backend/theano_backend.py:1282: UserWarning: DEPRECATION: the 'ds' parameter is not going to exist anymore as it is going to be replaced by the parameter 'ws'.
  mode='max')
/home/ubuntu/anaconda2/lib/python2.7/site-packages/keras/backend/theano_backend.py:1282: UserWarning: DEPRECATION: the 'st' parameter is not going to exist anymore as it is going to be replaced by the parameter 'stride'.
  mode='max')
/home/ubuntu/anaconda2/lib/python2.7/site-packages/keras/backend/theano_backend.py:1282: UserWarning: DEPRECATION: the 'padding' parameter is not going to exist anymore as it is going to be replaced by the parameter 'pad'.
  mode='max')


th
(None, 3, 450, 450) (None, 7)


In [17]:
# len(get_additional_layers(conv_layers))

In [18]:

model.fit_generator(train_flow, samples_per_epoch=train_flow.nb_sample, nb_epoch=5,
                                 validation_data=val_flow, nb_val_samples=val_flow.nb_sample)

Epoch 1/5
2912/2912 [==============================] - 387s - loss: 0.8304 - acc: 0.7229 - val_loss: 1.4179 - val_acc: 0.7125
Epoch 2/5
2912/2912 [==============================] - 386s - loss: 0.3997 - acc: 0.8685 - val_loss: 0.5869 - val_acc: 0.8550
Epoch 3/5
2912/2912 [==============================] - 385s - loss: 0.2542 - acc: 0.9076 - val_loss: 0.4036 - val_acc: 0.9000
Epoch 4/5
2912/2912 [==============================] - 386s - loss: 0.1808 - acc: 0.9382 - val_loss: 0.5357 - val_acc: 0.8525
Epoch 5/5
2912/2912 [==============================] - 385s - loss: 0.1245 - acc: 0.9557 - val_loss: 0.5183 - val_acc: 0.8800


In [9]:
# model.save_weights(path+"models/aug_cropped_1e3_5epch_val_acc_0.88.hd5")
model.load_weights(path+"models/aug_cropped_1e3_5epch_val_acc_0.88.hd5")


In [10]:
model.optimizer.lr = 1e-4
model.fit_generator(train_flow, samples_per_epoch=train_flow.nb_sample, nb_epoch=5,
                                 validation_data=val_flow, nb_val_samples=val_flow.nb_sample)

Epoch 1/5
2912/2912 [==============================] - 392s - loss: 0.0793 - acc: 0.9756 - val_loss: 0.1645 - val_acc: 0.9550
Epoch 2/5
2912/2912 [==============================] - 395s - loss: 0.0367 - acc: 0.9914 - val_loss: 0.1575 - val_acc: 0.9600
Epoch 3/5
2912/2912 [==============================] - 396s - loss: 0.0294 - acc: 0.9924 - val_loss: 0.1522 - val_acc: 0.9625
Epoch 4/5
2912/2912 [==============================] - 396s - loss: 0.0168 - acc: 0.9979 - val_loss: 0.1522 - val_acc: 0.9575
Epoch 5/5
2912/2912 [==============================] - 396s - loss: 0.0139 - acc: 0.9979 - val_loss: 0.1453 - val_acc: 0.9650


In [11]:
# model.save_weights(path+"models/aug_cropped_1e4_5epch_val_acc_0.9650.hd5")
model.load_weights(path+"models/aug_cropped_1e4_5epch_val_acc_0.9650.hd5")

In [28]:
model.optimizer.lr = 1e-5
model.fit_generator(train_flow, samples_per_epoch=train_flow.nb_sample, nb_epoch=5,
                                 validation_data=val_flow, nb_val_samples=val_flow.nb_sample)

Epoch 1/5
2912/2912 [==============================] - 393s - loss: 0.0199 - acc: 0.9948 - val_loss: 0.1406 - val_acc: 0.9650
Epoch 2/5
2912/2912 [==============================] - 395s - loss: 0.0196 - acc: 0.9952 - val_loss: 0.1389 - val_acc: 0.9650
Epoch 3/5
2912/2912 [==============================] - 396s - loss: 0.0149 - acc: 0.9955 - val_loss: 0.1380 - val_acc: 0.9650
Epoch 4/5
2912/2912 [==============================] - 395s - loss: 0.0184 - acc: 0.9966 - val_loss: 0.1382 - val_acc: 0.9650
Epoch 5/5
 288/2912 [=>............................] - ETA: 313s - loss: 0.0100 - acc: 0.9965

KeyboardInterrupt: 

model is overfitting without any changes in validation accuracy, rolling back to previous weights.

### Making augmentation heavier (not done)

In [20]:
gen = image.ImageDataGenerator(rotation_range=10, width_shift_range=0.1, zoom_range=0.1,
                                      channel_shift_range=5, height_shift_range=0.1, shear_range=0.1,
                                      fill_mode='nearest',horizontal_flip=True)

train_flow = gen.flow_from_directory(path+"train", target_size=target_size,
            class_mode="categorical", shuffle=True, batch_size=batch_size)

val_flow = image.ImageDataGenerator().flow_from_directory(path+"valid", target_size=target_size, batch_size=batch_size,
                                                   class_mode='categorical', shuffle=True)

Found 2912 images belonging to 7 classes.
Found 400 images belonging to 7 classes.


In [21]:
model.optimizer.lr = 1e-4
model.fit_generator(train_flow, samples_per_epoch=train_flow.nb_sample, nb_epoch=5,
                                 validation_data=val_flow, nb_val_samples=val_flow.nb_sample)

Epoch 1/5
2912/2912 [==============================] - 392s - loss: 0.0325 - acc: 0.9883 - val_loss: 0.1878 - val_acc: 0.9525
Epoch 2/5
2912/2912 [==============================] - 395s - loss: 0.0350 - acc: 0.9876 - val_loss: 0.1756 - val_acc: 0.9575
Epoch 3/5
 288/2912 [=>............................] - ETA: 314s - loss: 0.0166 - acc: 0.9965

KeyboardInterrupt: 

it's making the results worse, skipping for now 

### Make the original top 2 layers trainable
ToDO
After getting good enough accuracy on default VGG weights, it's time to allow the two top layers of original VGG to adapt

In [11]:
model.layers[-18:-10]

In [12]:
for layer in model.layers[-18:-14]:
    layer.trainable = True    

In [19]:
model_path = path+"models/" + 'fcn_aug_vggtunetop2_{val_acc:.2f}val_acc-{val_loss:.2f}-loss_{epoch}epoch.h5' 
ckpt = ModelCheckpoint(filepath=model_path, monitor='val_loss',
                           save_best_only=True, save_weights_only=True)


model.optimizer.lr = 1e-5

model.fit_generator(train_flow, samples_per_epoch=train_flow.nb_sample, nb_epoch=3,
                                 validation_data=val_flow, nb_val_samples=val_flow.nb_sample, callbacks = [ckpt,early_stopping])

Epoch 1/3
2912/2912 [==============================] - 363s - loss: 0.0273 - acc: 0.9924 - val_loss: 0.1497 - val_acc: 0.9650
Epoch 2/3
2912/2912 [==============================] - 364s - loss: 0.0194 - acc: 0.9952 - val_loss: 0.1495 - val_acc: 0.9675
Epoch 3/3
2912/2912 [==============================] - 364s - loss: 0.0207 - acc: 0.9931 - val_loss: 0.1512 - val_acc: 0.9675


best of default vgg:  loss: 0.0139 - acc: 0.9979 - val_loss: 0.1453 - val_acc: 0.9650

In [22]:
model.optimizer.lr = 1e-6

model.fit_generator(train_flow, samples_per_epoch=train_flow.nb_sample, nb_epoch=5,
                                 validation_data=val_flow, nb_val_samples=val_flow.nb_sample, callbacks = [ckpt,early_stopping])

Epoch 1/5
2912/2912 [==============================] - 363s - loss: 0.0375 - acc: 0.9883 - val_loss: 0.1560 - val_acc: 0.9650
Epoch 2/5
2912/2912 [==============================] - 364s - loss: 0.0340 - acc: 0.9883 - val_loss: 0.1543 - val_acc: 0.9675
Epoch 3/5
2912/2912 [==============================] - 364s - loss: 0.0267 - acc: 0.9945 - val_loss: 0.1560 - val_acc: 0.9650
Epoch 4/5
2912/2912 [==============================] - 364s - loss: 0.0366 - acc: 0.9876 - val_loss: 0.1592 - val_acc: 0.9650


### Test Augmentation

In [32]:
test_datagen = gen
num_iterations = 5
preds = np.zeros((1000, 7)) #1000 test images, 7 fish types 

for i in range(num_iterations):
    test_gen = test_datagen.flow_from_directory(path+"test", target_size=target_size, batch_size=batch_size,
                                                        class_mode=None, shuffle=False)
    preds += model.predict_generator(test_gen, val_samples=1000)
    print ("finished predicting round {}".format(i))
    
    

Found 1000 images belonging to 1 classes.
finished predicting round 0
Found 1000 images belonging to 1 classes.
finished predicting round 1
Found 1000 images belonging to 1 classes.
finished predicting round 2
Found 1000 images belonging to 1 classes.
finished predicting round 3
Found 1000 images belonging to 1 classes.
finished predicting round 4


Stage 2

In [15]:
test_datagen = gen
num_iterations = 5
preds = np.zeros((len(test_filenames), 7)) #1000 test images, 7 fish types 

for i in range(num_iterations):
    test_gen = test_datagen.flow_from_directory(path+"test", target_size=target_size, batch_size=batch_size,
                                                        class_mode=None, shuffle=False)
    preds += model.predict_generator(test_gen, val_samples=len(test_filenames))
    save_array(path+"results/preds_light_aug_cropped_v1_part{}.dat".format(i+1),preds)
    
    print ("finished predicting round {}".format(i))
    
    

Found 12153 images belonging to 1 classes.
finished predicting round 0
Found 12153 images belonging to 1 classes.
finished predicting round 1
Found 12153 images belonging to 1 classes.
finished predicting round 2
Found 12153 images belonging to 1 classes.
finished predicting round 3
Found 12153 images belonging to 1 classes.
finished predicting round 4


In [16]:
preds[0:1]

array([[  3.7026e+00,   1.6729e-04,   4.6911e-03,   2.0469e-04,   7.4383e-01,   1.5433e-04,
          5.4837e-01]])

stage 1 preds[0:1]:

array([[  4.9990e+00,   1.4059e-04,   4.9461e-06,   3.9640e-06,   7.3655e-04,   1.4233e-04,
          7.2796e-06]])

In [17]:
averaged_preds = preds / num_iterations

In [25]:
# save_array(path+"results/preds_light_aug_cropped_v1.dat",averaged_preds)
save_array(path+"results/stg2_preds_light_aug_cropped_v1.dat",averaged_preds)


###  Resnet (not done)
Having problems with loss values, skipping for now in favor of a simple model with dense layers on top

In [57]:
from resnet50 import Resnet50

resnet = Resnet50()


In [58]:

resmodel = resnet.model 

# removing top layer that was designed for imagenet (1000 classes)
resmodel.layers.pop()

for layer in resmodel.layers: 
    layer.trainable=False


# classifying 7 types of fish 
m = Dense(7, activation='softmax')(resmodel.layers[-1].output)
resmodel= Model(resmodel.input, m)

resmodel.compile(optimizer=Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

# RMSprop(lr=0.1)

In [59]:
# creating new flow generatros with size (224,224)
res_train_flow = gen.flow_from_directory(path+"train", target_size=(224,224),
            class_mode="categorical", shuffle=True, batch_size=batch_size)

res_val_flow = image.ImageDataGenerator().flow_from_directory(path+"valid", target_size=(224,224), batch_size=batch_size,
                                                   class_mode='categorical', shuffle=True)




resmodel.fit_generator(res_train_flow, samples_per_epoch=res_train_flow.nb_sample, nb_epoch=5,
                                 validation_data=res_val_flow, nb_val_samples=res_val_flow.nb_sample)

Found 2912 images belonging to 7 classes.
Found 400 images belonging to 7 classes.
Epoch 1/5
2912/2912 [==============================] - 81s - loss: nan - acc: 0.5251 - val_loss: nan - val_acc: 0.4750
Epoch 2/5
1312/2912 [============>.................] - ETA: 40s - loss: nan - acc: 0.5244

KeyboardInterrupt: 

i get NaN for loss, skipping for now

### VGG with Dense layer on top

Super mild augmentation,
0.04 improvement!

In [5]:
from vgg16bn import Vgg16BN

# vgg_dense = Vgg16BN(size=target_size).model

# vgg_dense.pop()
# print (vgg_dense.input_shape, vgg_dense.output_shape)

# vgg640.compile(Adam(), 'categorical_crossentropy', metrics=['accuracy'])

# conv_layers,fc_layers = split_at(vgg640, Convolution2D)
# conv_model = Sequential(conv_layers)


In [6]:
def get_bn_layers(p):
    return [
        MaxPooling2D(input_shape=conv_layers[-1].output_shape[1:]),
        BatchNormalization(axis=1),
        Dropout(p/3),
        Flatten(),
        Dense(2048, activation='relu'),
        BatchNormalization(),
        Dropout(p/2),
        Dense(2048, activation='relu'),
        BatchNormalization(),
        Dropout(p),
        Dense(7, activation='softmax')
    ]

In [7]:
train_datagen = image.ImageDataGenerator(
        shear_range=0.05,
        zoom_range=0.05,
        rotation_range=7,
        width_shift_range=0.07,
        height_shift_range=0.07,
    channel_shift_range=5,
        horizontal_flip=True)


# set shuffle=False if pre-computing

# aug_trn_batches = train_datagen.flow_from_directory(path+'train', target_size=target_size,
#            class_mode='categorical', shuffle=False, batch_size=batch_size*2)


# aug_val_batches = image.ImageDataGenerator().flow_from_directory(path+"valid", target_size=target_size, batch_size=batch_size*2,
#                                                    class_mode='categorical', shuffle=False)

In [8]:
#precomputing for faster computation 

In [11]:
vgg_dense = Vgg16BN(size=target_size).model

vgg_dense.pop()
print (vgg_dense.input_shape, vgg_dense.output_shape)


conv_layers,fc_layers = split_at(vgg_dense, Convolution2D)

(None, 3, 450, 450) (None, 512, 28, 28)


In [25]:
da_conv_featx2 = vgg_dense.predict_generator(aug_trn_batches, aug_trn_batches.nb_sample*2)


In [26]:
da_conv_featx2.shape

(5824, 512, 28, 28)

In [27]:
save_array(path+"results/da_conv_featx2.dat",da_conv_featx2)

In [15]:
conv_trn_feat = load_array(path+'results/conv_trn_450.dat')

In [28]:
conv_trn_feat.shape

(2912, 512, 28, 28)

In [29]:
da_conv_feat = np.concatenate([conv_trn_feat, da_conv_featx2])

In [30]:
da_trn_labels = np.concatenate([trn_labels]*3)
conv_val_feat = load_array(path+'results/conv_val_450.dat')


In [48]:
# memory cleaning
# del conv_val_feat, da_conv_feat,conv_trn_feat,da_conv_featx2

In [12]:
dense_model = Sequential(get_bn_layers(0.6))
dense_model.compile(Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])



In [40]:
dense_model.fit(da_conv_feat, da_trn_labels, batch_size=64, nb_epoch=5, 
             validation_data=(conv_val_feat, val_labels))

Train on 8736 samples, validate on 400 samples
Epoch 1/5
8736/8736 [==============================] - 33s - loss: 0.8483 - acc: 0.8156 - val_loss: 0.5322 - val_acc: 0.9250
Epoch 2/5
8736/8736 [==============================] - 33s - loss: 0.0996 - acc: 0.9752 - val_loss: 0.3817 - val_acc: 0.9575
Epoch 3/5
8736/8736 [==============================] - 33s - loss: 0.0419 - acc: 0.9899 - val_loss: 0.3930 - val_acc: 0.9625
Epoch 4/5
8736/8736 [==============================] - 33s - loss: 0.0353 - acc: 0.9920 - val_loss: 0.3403 - val_acc: 0.9675
Epoch 5/5
8736/8736 [==============================] - 33s - loss: 0.0326 - acc: 0.9930 - val_loss: 0.3397 - val_acc: 0.9575


In [42]:
dense_model.optimizer.lr = 1e-5
dense_model.fit(da_conv_feat, da_trn_labels, batch_size=64, nb_epoch=3, 
             validation_data=(conv_val_feat, val_labels))

Train on 8736 samples, validate on 400 samples
Epoch 1/3
8736/8736 [==============================] - 33s - loss: 0.0347 - acc: 0.9910 - val_loss: 0.3026 - val_acc: 0.9625
Epoch 2/3
8736/8736 [==============================] - 33s - loss: 0.0200 - acc: 0.9938 - val_loss: 0.2869 - val_acc: 0.9650
Epoch 3/3
8736/8736 [==============================] - 33s - loss: 0.0382 - acc: 0.9904 - val_loss: 0.3190 - val_acc: 0.9525


In [43]:
dense_model.optimizer.lr = 1e-7
dense_model.fit(da_conv_feat, da_trn_labels, batch_size=64, nb_epoch=3, 
             validation_data=(conv_val_feat, val_labels))

Train on 8736 samples, validate on 400 samples
Epoch 1/3
8736/8736 [==============================] - 34s - loss: 0.0237 - acc: 0.9942 - val_loss: 0.3196 - val_acc: 0.9675
Epoch 2/3
8736/8736 [==============================] - 33s - loss: 0.0294 - acc: 0.9936 - val_loss: 0.2976 - val_acc: 0.9625
Epoch 3/3
8736/8736 [==============================] - 33s - loss: 0.0234 - acc: 0.9937 - val_loss: 0.2968 - val_acc: 0.9625


In [13]:
# dense_model.save_weights(path+"models/dense_precomputed_aug_1e7_val_acc_0.9625.h5")

dense_model.load_weights(path+"models/dense_precomputed_aug_1e7_val_acc_0.9625.h5")


#### Making predictions
    

In [16]:
predictions = np.zeros(shape=(1000, 7))

nb_aug = 5
for aug in range(nb_aug):
    
    test_batches = train_datagen.flow_from_directory(path+"test", target_size=target_size,
                class_mode=None, shuffle=False, batch_size=32)


    conv_test_feat = vgg_dense.predict_generator(test_batches, 1000)
    predictions += dense_model.predict(conv_test_feat, batch_size=32)
    
    print ("finished prediction round ", aug+1)
predictions /= nb_aug

Found 1000 images belonging to 1 classes.
finished prediction round  1
Found 1000 images belonging to 1 classes.
finished prediction round  2
Found 1000 images belonging to 1 classes.
finished prediction round  3
Found 1000 images belonging to 1 classes.
finished prediction round  4
Found 1000 images belonging to 1 classes.
finished prediction round  5


In [18]:
save_array(path+"results/preds_dense_precomputed_aug_1e7_val_acc_0.9625.dat",predictions)

Stage 2 predictions:

In [14]:
predictions = np.zeros(shape=(len(test_filenames), 7))

nb_aug = 5
for aug in range(nb_aug):
    
    test_batches = train_datagen.flow_from_directory(path+"test", target_size=target_size,
                class_mode=None, shuffle=False, batch_size=32)


    conv_test_feat = vgg_dense.predict_generator(test_batches, len(test_filenames))
    predictions += dense_model.predict(conv_test_feat, batch_size=32)
    
    print ("finished prediction round ", aug+1)
    
    save_array(path+"results/stg2_croppreds_dense_precomputed_aug_part{}.dat".format(aug+1),predictions)

    
    
    
predictions /= nb_aug

Found 12153 images belonging to 1 classes.
finished prediction round  1
Found 12153 images belonging to 1 classes.
finished prediction round  2
Found 12153 images belonging to 1 classes.
finished prediction round  3
Found 12153 images belonging to 1 classes.
finished prediction round  4
Found 12153 images belonging to 1 classes.
finished prediction round  5


In [15]:
save_array(path+"results/stg2_preds_dense_precomputed_aug_1e7_val_acc_0.9625.dat",predictions)

In [17]:
predictions.shape

(12153, 7)

In [18]:
predictions[0:2]

array([[  2.7109e-01,   9.4312e-04,   5.9165e-03,   9.3670e-03,   1.9749e-01,   3.4832e-04,
          5.1485e-01],
       [  9.8965e-01,   7.4218e-05,   9.4611e-05,   4.9753e-06,   1.0168e-02,   3.6449e-07,
          5.6005e-06]])

### Vanilla VGG with Dense layer on top

vanilla: no aug, precomputed 
Result: worse by 0.05 points


In [24]:
trn = get_data(path+'train', target_size)
val = get_data(path+'valid', target_size)

Found 2912 images belonging to 7 classes.
Found 400 images belonging to 7 classes.


In [25]:
save_array(path+'results/trn_450.dat', trn)
save_array(path+'results/trn_450.dat', val)

In [30]:
vgg450 = Vgg16BN(target_size).model
vgg450.pop()
print (vgg450.input_shape, vgg450.output_shape)
vgg450.compile(Adam(), 'categorical_crossentropy', metrics=['accuracy'])

(None, 3, 450, 450) (None, 512, 28, 28)


In [31]:
conv_val_feat = vgg450.predict(val, batch_size=32, verbose=1)
conv_trn_feat = vgg450.predict(trn, batch_size=32, verbose=1)

2912/2912 [==============================] - 314s   


In [32]:
save_array(path+'results/conv_val_450.dat', conv_val_feat)
save_array(path+'results/conv_trn_450.dat', conv_trn_feat)

In [33]:
test = get_data(path+'test', target_size)
save_array(path+'results/test_450.dat', test)

conv_test_feat = vgg450.predict(test, batch_size=32, verbose=1)
save_array(path+'results/conv_test_450.dat', conv_test_feat)

Found 1000 images belonging to 1 classes.
1000/1000 [==============================] - 107s   


In [52]:
conv_layers,fc_layers = split_at(vgg450, Convolution2D)
bn_model = Sequential(get_bn_layers(p))
bn_model.compile(Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

In [53]:
bn_model.fit(conv_trn_feat, trn_labels, batch_size=batch_size, nb_epoch=2, 
             validation_data=(conv_val_feat, val_labels))

Train on 2912 samples, validate on 400 samples
Epoch 1/2
2912/2912 [==============================] - 13s - loss: 0.7027 - acc: 0.8156 - val_loss: 0.2767 - val_acc: 0.9375
Epoch 2/2
2912/2912 [==============================] - 13s - loss: 0.0685 - acc: 0.9787 - val_loss: 0.2963 - val_acc: 0.9350


In [55]:
# bn_model.save_weights(path+"models/vanilla_vgg_precompute_1e3_0.9475.h5")
bn_model.load_weights(path+"models/vanilla_vgg_precompute_1e3_0.9475.h5")

In [54]:
bn_model.optimizer.lr = 1e-7
bn_model.fit(conv_trn_feat, trn_labels, batch_size=batch_size, nb_epoch=3, 
             validation_data=(conv_val_feat, val_labels))

Train on 2912 samples, validate on 400 samples
Epoch 1/3
2912/2912 [==============================] - 13s - loss: 0.0549 - acc: 0.9866 - val_loss: 0.2333 - val_acc: 0.9500
Epoch 2/3
2912/2912 [==============================] - 13s - loss: 0.0404 - acc: 0.9870 - val_loss: 0.2538 - val_acc: 0.9450
Epoch 3/3
2912/2912 [==============================] - 13s - loss: 0.0554 - acc: 0.9849 - val_loss: 0.3063 - val_acc: 0.9300


In [56]:
preds_vanilla = bn_model.predict(conv_test_feat)


In [58]:
save_array(path+"results/preds_dense_vanilla_1e3.dat",preds_vanilla)

### Submission

In [23]:
import pandas as pd

classes = ['ALB', 'BET', 'DOL', 'LAG','NoF', 'OTHER', 'SHARK', 'YFT']
# nofish_prob = load_array("data/fishnofish/results/preds_640_loss_0.0244_acc_0.9913_val_loss_0.2086_val_acc_0.9778.dat")
# nofish_prob = load_array("data/fishnofish/results/pred_stg1_fullconv_aug_no_pseudo.dat")
# nofish_prob = load_array("data/fishnofish/results/final_stg1_weights_ensumble.dat")

nofish_prob = load_array("data/fishnofish/results/final_stg1_weights_ensumble_with_pseudo.dat")



nofish_prob = nofish_prob[:, 1] #nofish class


In [24]:

preds = load_array(path+"results/preds_light_aug_cropped_v1.dat")
# preds_vanilla = load_array(path+"results/preds_dense_vanilla_1e3.dat") >> made it worse by 0.7 point
preds_denseaug = load_array(path+"results/preds_dense_precomputed_aug_1e7_val_acc_0.9625.dat")

subm_name = path+'submit/preds_cropped_v1_denseaugprecompute.gz'



In [25]:
preds+= preds_denseaug
preds /= 2.0

In [26]:
def weight_predictions(predictions):
    """Weights predictions based on probability image contains a fish as predicted by fish detector model"""
    no_fish = predictions[:, 4]
    fish = np.delete(predictions, 4, axis=1)

    weights = -1. * (no_fish - 1.)
    weights = weights.reshape(1000, 1)

    fish = weights * fish
    preds = np.insert(fish, 4, no_fish, axis=1)

    return preds


In [27]:
with_nofish_probs = np.insert(preds, 4, nofish_prob, axis=1)
weighted_preds = weight_predictions(with_nofish_probs)

In [28]:
f = 0.85#0.82
subm = np.clip(weighted_preds, (1-f)/7, f)

In [29]:
submission = pd.DataFrame(subm, columns=classes)
submission.insert(0, 'image', raw_test_filenames)
submission.head()

,image,ALB,BET,DOL,LAG,NoF,OTHER,SHARK,YFT
0,img_06237.jpg,0.850000,0.021429,0.021429,0.021429,0.021429,0.021429,0.021429,0.021429
1,img_06893.jpg,0.299337,0.021429,0.021429,0.021429,0.021429,0.022149,0.021429,0.638110
2,img_02082.jpg,0.360107,0.021429,0.021429,0.021429,0.582517,0.021429,0.021429,0.054423
3,img_06261.jpg,0.850000,0.021429,0.021429,0.021429,0.021429,0.021429,0.021429,0.107314
4,img_03628.jpg,0.850000,0.021429,0.021429,0.021429,0.021429,0.021429,0.021429,0.021429


In [30]:
submission.to_csv(subm_name, index=False, compression='gzip')
FileLink(subm_name)

/home/ubuntu/fish/data/cropped_nof_excl/submit/preds_cropped_v1_denseaugprecompute.gz